In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import re
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import sys
import time

# import utils
sys.path.append("../../../utils")
from plotting_utils import *
from classify_utils import *

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.autolayout'] = False

In [2]:
sns.set(**PAPER_PRESET)
fontsize = PAPER_FONTSIZE

In [3]:
np.random.seed(2019)

## functions

In [4]:
def min_biotype(row):
    if row.cleaner_gene_biotype == "protein_coding":
        return "mRNA"
    else:
        return "lncRNA"

In [5]:
def min_hit(row):
    if row.is_hit == "stringent hit":
        return "hit"
    elif row.is_hit == "lenient hit":
        return "lenient"
    else:
        return "no hit"

## variables

In [6]:
# features
feature_dir = "../../../misc/08__model_features"
splicing_f = "%s/gene_splicing_efficiency.with_DIGIT.txt" % feature_dir
gc_f = "%s/gc_content.with_DIGIT.txt" % feature_dir
n_tss_f = "%s/n_tss_within_100bp.with_DIGIT.txt" % feature_dir
n_enh_f = "%s/n_enhancers_within_1mb.with_DIGIT.txt" % feature_dir
closest_enh_f = "%s/closest_enh_to_TSS.with_DIGIT.txt" % feature_dir
prom_cons_f = "%s/promoter_conservation.500buff.with_DIGIT.txt" % feature_dir
exon_cons_f = "%s/exon_conservation.with_DIGIT.txt" % feature_dir
dna_len_f = "%s/transcript_length.with_DIGIT.txt" % feature_dir
rna_len_f = "%s/transcript_length_RNA.with_DIGIT.txt" % feature_dir
n_exons_f = "%s/n_exons_per_transcript.with_DIGIT.txt" % feature_dir

In [7]:
# gwas file
gwas_dir = "../../../misc/06__gwas"
closest_endo_f = "%s/transcript_coords.closest_endo_cancer_snp.with_DIGIT.bed" % gwas_dir

In [8]:
gene_map_f = "../../../misc/00__gene_metadata/gencode.v25lift37.GENE_ID_TRANSCRIPT_ID_MAP.with_DIGIT.fixed.txt"

In [9]:
hits_f = "../../../data/02__screen/02__enrichment_data/enrichment_values.with_rna_seq.txt"

## 1. import data

In [10]:
splicing = pd.read_table(splicing_f)

In [11]:
gc = pd.read_table(gc_f, header=None)
gc.columns = ["transcript_id", "gc"]

In [12]:
n_tss = pd.read_table(n_tss_f, delim_whitespace=True, header=None)
n_tss.columns = ["n_tss", "transcript_id"]

In [13]:
n_enh = pd.read_table(n_enh_f, delim_whitespace=True, header=None)
n_enh.columns = ["n_enh", "transcript_id"]

In [14]:
closest_enh = pd.read_table(closest_enh_f, header=None)
closest_enh.columns = ["transcript_id", "enh_id", "enh_dist"]

In [15]:
prom_cons = pd.read_table(prom_cons_f)

In [16]:
exon_cons = pd.read_table(exon_cons_f)

In [17]:
dna_len = pd.read_table(dna_len_f, header=None)
dna_len.columns = ["transcript_id", "dna_len"]

In [18]:
rna_len = pd.read_table(rna_len_f, header=None)
rna_len.columns = ["transcript_id", "rna_len"]

In [19]:
n_exons = pd.read_table(n_exons_f, header=None)
n_exons.columns = ["n_exons", "gene_id", "transcript_id"]

In [20]:
closest_endo = pd.read_table(closest_endo_f, sep="\t", header=None)
closest_endo.columns = ["chr", "start", "end", "transcript_id", "snp_chr", "snp_start", "snp_end",
                        "closest_endo_snp_id", "closest_endo_snp_disease", "closest_endo_snp_distance"]

In [21]:
gene_map = pd.read_table(gene_map_f, header=None)
gene_map.columns = ["gene_id", "transcript_id"]

In [22]:
hits = pd.read_table(hits_f)

## 2. join transcript-level data w/ gene id

In [23]:
print(len(gc))
gc = gc.merge(gene_map, on="transcript_id")
print(len(gc))

200140
200140


In [24]:
print(len(n_tss))
n_tss = n_tss.merge(gene_map, on="transcript_id")
print(len(n_tss))

94934
94934


In [25]:
print(len(n_enh))
n_enh = n_enh.merge(gene_map, on="transcript_id")
print(len(n_enh))

199413
199413


In [26]:
print(len(closest_enh))
closest_enh = closest_enh.merge(gene_map, on="transcript_id")
print(len(closest_enh))

200148
200148


In [27]:
print(len(prom_cons))
prom_cons = prom_cons.merge(gene_map, left_on="name", right_on="transcript_id")
print(len(prom_cons))

200140
200140


In [28]:
print(len(exon_cons))
exon_cons = exon_cons.merge(gene_map, left_on="name", right_on="transcript_id")
print(len(exon_cons))

1185571
1185571


In [29]:
print(len(dna_len))
dna_len = dna_len.merge(gene_map, on="transcript_id")
print(len(dna_len))

200140
200140


In [30]:
print(len(rna_len))
rna_len = rna_len.merge(gene_map, on="transcript_id")
print(len(rna_len))

200140
200140


In [31]:
print(len(closest_endo))
closest_endo = closest_endo.merge(gene_map, on="transcript_id")
print(len(closest_endo))

255868
255868


## 3. aggregate features to gene level

In [32]:
gc_gene = gc.groupby("gene_id")["gc"].agg("mean").reset_index()
print(len(gc_gene))

60253


In [33]:
n_tss_gene = n_tss.groupby("gene_id")["n_tss"].agg("max").reset_index()
print(len(n_tss_gene))

23065


In [34]:
n_enh_gene = n_enh.groupby("gene_id")["n_enh"].agg("max").reset_index()
print(len(n_enh_gene))

59781


In [35]:
closest_enh_gene = closest_enh.groupby("gene_id")["enh_dist"].agg("min").reset_index()
print(len(closest_enh_gene))

60253


In [36]:
prom_cons_gene = prom_cons.groupby("gene_id")["median"].agg("max").reset_index()
prom_cons_gene.columns = ["gene_id", "prom_cons"]
print(len(prom_cons_gene))

60253


In [37]:
exon_cons_tx = exon_cons.groupby(["name", "gene_id"])["median"].agg("mean").reset_index()
exon_cons_gene = exon_cons_tx.groupby("gene_id")["median"].agg("max").reset_index()
exon_cons_gene.columns = ["gene_id", "exon_cons"]
print(len(exon_cons_gene))

60253


In [38]:
dna_len_gene = dna_len.groupby("gene_id")["dna_len"].agg("max").reset_index()
print(len(dna_len_gene))

60253


In [39]:
rna_len_gene = rna_len.groupby("gene_id")["rna_len"].agg("max").reset_index()
print(len(rna_len_gene))

60253


In [40]:
n_exons_gene = n_exons.groupby("gene_id")["n_exons"].agg("max").reset_index()
print(len(n_exons_gene))

60253


In [41]:
# same thing for RNA-seq data: sum up transcript expression levels
endo_exp = hits[["gene_id", "hESC_mean", "endo_mean"]].groupby("gene_id")[["hESC_mean", "endo_mean"]].agg("sum").reset_index()
print(len(endo_exp))

8167


In [42]:
# take transcript w/ maximum logfc expression
endo_fc = hits[["gene_id", "endo_hESC_abslog2fc"]].groupby("gene_id")["endo_hESC_abslog2fc"].agg("max").reset_index()
print(len(endo_fc))

8167


In [43]:
# need to also do this for gwas
closest_endo_gene = closest_endo[["gene_id", 
                                  "closest_endo_snp_distance"]].groupby("gene_id")["closest_endo_snp_distance"].agg("min").reset_index()
closest_endo_gene = closest_endo_gene.merge(closest_endo[["gene_id", "closest_endo_snp_distance", 
                                                          "closest_endo_snp_id", "closest_endo_snp_disease"]],
                                            on=["gene_id", 
                                                "closest_endo_snp_distance"]).drop_duplicates(subset=["gene_id", 
                                                                                                      "closest_endo_snp_distance"])
print(len(closest_endo_gene))

60253


## 4. merge all genomic features into 1 dataframe

In [44]:
data = splicing.merge(gc_gene, on="gene_id", how="outer")
print(len(data))

60253


In [45]:
data = data.merge(n_tss_gene, on="gene_id", how="left").merge(n_enh_gene, on="gene_id", how="left")
print(len(data))

60253


In [46]:
data = data.merge(closest_enh_gene, on="gene_id", how="left").merge(prom_cons_gene, on="gene_id", how="left")
print(len(data))

60253


In [47]:
data = data.merge(exon_cons_gene, on="gene_id", how="left").merge(dna_len_gene, on="gene_id", how="left")
print(len(data))

60253


In [48]:
data = data.merge(rna_len_gene, on="gene_id", how="left").merge(n_exons_gene, on="gene_id", how="left")
print(len(data))

60253


In [49]:
data = data.merge(closest_endo_gene[["gene_id", "closest_endo_snp_distance", "closest_endo_snp_id",
                                     "closest_endo_snp_disease"]], on="gene_id", how="left")
print(len(data))

60253


In [50]:
# for n tss and n enh, NAs do not mean lack of data but mean 0 -- so replace NAs in these cols with 0
data["n_tss"] = data["n_tss"].fillna(0)
data["n_enh"] = data["n_enh"].fillna(0)

In [51]:
data["short_gene_id"] = data["gene_id"].str.split("_", expand=True)[0]
data["shorter_gene_id"] = data["short_gene_id"].str.split(".", expand=True)[0]
data["minimal_biotype"] = data.apply(min_biotype, axis=1)
data.minimal_biotype.value_counts()

lncRNA    40121
mRNA      20132
Name: minimal_biotype, dtype: int64

In [52]:
# remove bad biotypes we don't care about (like pseudogenes) which will have a null gene_name value
data_filt = data[~pd.isnull(data["gene_name"])]
len(data_filt)

33856

## 5. create df including only genes in screen + include endo RNAseq features

In [53]:
genes_in_screen = hits["gene_id"].unique()
len(genes_in_screen)

8167

In [54]:
hits["min_hit"] = hits.apply(min_hit, axis=1)
hits.min_hit.value_counts()

no hit     11196
lenient     1428
hit           73
Name: min_hit, dtype: int64

In [55]:
gene_hit_status = hits[["gene_id", "gene_name", "endo_ctrl_val", "cleaner_gene_biotype",
                        "min_hit"]].sort_values(by=["gene_id", "min_hit"]).drop_duplicates(subset="gene_id", 
                                                                                           keep="first")
gene_hit_status.head()

,gene_id,gene_name,endo_ctrl_val,cleaner_gene_biotype,min_hit
118,DIGIT,DIGIT,False,promoter_overlap,hit
1680,ENSG00000005206.12,SPPL2B,False,promoter_overlap,no hit
1205,ENSG00000008311.14_1,AASS,False,protein_coding,no hit
2483,ENSG00000010278.12_2,CD9,False,protein_coding,no hit
9453,ENSG00000013297.10_2,CLDN11,False,protein_coding,no hit


In [56]:
print(len(data))
df_screen = data.merge(gene_hit_status, on=["gene_id", "gene_name", "cleaner_gene_biotype"])
print(len(df_screen))
df_screen.head()

60253
8167


,gene_id,IMR-90_eff_mean,IMR-90_exp_mean,IMR-90_eff_ratio,K562_eff_mean,K562_exp_mean,K562_eff_ratio,HT1080_eff_mean,HT1080_exp_mean,HT1080_eff_ratio,...,rna_len,n_exons,closest_endo_snp_distance,closest_endo_snp_id,closest_endo_snp_disease,short_gene_id,shorter_gene_id,minimal_biotype,endo_ctrl_val,min_hit
0,ENSG00000243485.4_2,0.000000,0.000000,0.000000,0.038730,0.115228,0.004463,0.000000,0.038927,0.000000,...,712,3,863464,rs13303010,pancreatic carcinoma,ENSG00000243485.4,ENSG00000243485,lncRNA,False,no hit
1,ENSG00000237613.2_1,0.075812,0.089398,0.006777,0.014606,0.032696,0.000478,0.010058,0.010983,0.000110,...,1187,3,858492,rs13303010,pancreatic carcinoma,ENSG00000237613.2,ENSG00000237613,lncRNA,False,lenient
2,ENSG00000238009.6_2,0.114828,0.131461,0.015095,0.688701,2.415708,1.663702,0.206727,0.313033,0.064712,...,2748,4,760850,rs13303010,pancreatic carcinoma,ENSG00000238009.6,ENSG00000238009,lncRNA,False,no hit
3,ENSG00000239945.1_2,0.000000,0.469485,0.000000,0.002306,1.325207,0.003055,0.000000,0.730441,0.000000,...,1319,2,803468,rs13303010,pancreatic carcinoma,ENSG00000239945.1,ENSG00000239945,lncRNA,False,no hit
4,ENSG00000239906.1_1,0.197971,0.327736,0.064882,0.623117,2.832431,1.764937,0.000000,0.016915,0.000000,...,323,2,754234,rs13303010,pancreatic carcinoma,ENSG00000239906.1,ENSG00000239906,lncRNA,False,no hit


In [57]:
df_screen = df_screen.merge(endo_exp, on="gene_id").merge(endo_fc, on="gene_id")
len(df_screen)

8167

In [58]:
df_screen[df_screen["min_hit"] == "hit"].minimal_biotype.value_counts()

lncRNA    43
mRNA       9
Name: minimal_biotype, dtype: int64

In [59]:
tmp = df_screen[df_screen["min_hit"] == "hit"][["gene_name", "minimal_biotype", "endo_ctrl_val", "cleaner_gene_biotype"]]
tmp

,gene_name,minimal_biotype,endo_ctrl_val,cleaner_gene_biotype
68,RP3-510D11.2,lncRNA,False,transcript_overlap
107,RP5-1056L3.1,lncRNA,False,promoter_overlap
255,FOXD3-AS1,lncRNA,False,transcript_overlap
331,RP11-421L21.3,lncRNA,False,promoter_overlap
351,LAMTOR5-AS1,lncRNA,False,transcript_overlap
516,RP11-533E19.7,lncRNA,False,promoter_overlap
622,MIXL1,mRNA,True,protein_coding
965,AC011753.5,lncRNA,False,transcript_overlap
1303,RP11-222K16.2,lncRNA,False,intergenic
1304,EOMES,mRNA,True,protein_coding


In [60]:
tmp.groupby(["minimal_biotype", "endo_ctrl_val"])["gene_name"].agg("count")

minimal_biotype  endo_ctrl_val
lncRNA           False            43
mRNA             False             3
                 True              6
Name: gene_name, dtype: int64

## 6. write final files

### general features for all genes

In [61]:
meta_cols = ["gene_id", "gene_name", "csf", "cleaner_gene_biotype", "minimal_biotype"]
sub_feature_cols = ['max_eff', 'max_exp', 'gc', 'n_tss', 'n_enh', 'enh_dist', 'prom_cons',
                    'exon_cons', 'dna_len', 'rna_len', 'n_exons']

In [62]:
all_cols = meta_cols + sub_feature_cols

In [63]:
supp = data_filt[all_cols]
supp.head()

,gene_id,gene_name,csf,cleaner_gene_biotype,minimal_biotype,max_eff,max_exp,gc,n_tss,n_enh,enh_dist,prom_cons,exon_cons,dna_len,rna_len,n_exons
2,ENSG00000243485.4_2,MIR1302-2,lncRNA_good_csf,intergenic,lncRNA,0.208126,0.559674,0.545147,0.0,18.0,809376,0.00,0.000000,1543,712,3
3,ENSG00000237613.2_1,FAM138A,lncRNA_good_csf,intergenic,lncRNA,0.154032,0.222609,0.465565,0.0,18.0,803661,0.00,0.006667,1527,1187,3
6,ENSG00000186092.4_1,OR4F5,protein_coding,protein_coding,mRNA,0.023544,0.050019,0.571895,0.0,20.0,770651,0.00,0.000000,917,918,1
7,ENSG00000238009.6_2,RP11-34P13.7,lncRNA_good_csf,transcript_overlap,lncRNA,0.688701,2.415708,0.537288,0.0,26.0,706019,0.38,0.505000,36987,2748,4
8,ENSG00000239945.1_2,RP11-34P13.8,lncRNA_good_csf,transcript_overlap,lncRNA,0.027553,2.987223,0.525398,0.0,21.0,748637,0.11,0.140000,1554,1319,2


In [64]:
supp.to_csv("../../../data/03__features/SuppTable_AllFeatures.txt", sep="\t", index=False)

### + endo features for screen genes

In [65]:
meta_cols = ["gene_id", "gene_name", "csf", "cleaner_gene_biotype", "minimal_biotype", "endo_ctrl_val", "min_hit"]
sub_feature_cols = ['max_eff', 'max_exp', 'gc', 'n_tss', 'n_enh', 'enh_dist', 'prom_cons',
                    'exon_cons', 'dna_len', 'rna_len', 'n_exons', 'hESC_mean', 'endo_mean', 'endo_hESC_abslog2fc',
                    'closest_endo_snp_distance', 'closest_endo_snp_id', 'closest_endo_snp_disease']

In [66]:
all_cols = meta_cols + sub_feature_cols

In [67]:
supp = df_screen[all_cols]
supp.head()

,gene_id,gene_name,csf,cleaner_gene_biotype,minimal_biotype,endo_ctrl_val,min_hit,max_eff,max_exp,gc,...,exon_cons,dna_len,rna_len,n_exons,hESC_mean,endo_mean,endo_hESC_abslog2fc,closest_endo_snp_distance,closest_endo_snp_id,closest_endo_snp_disease
0,ENSG00000243485.4_2,MIR1302-2,lncRNA_good_csf,intergenic,lncRNA,False,no hit,0.208126,0.559674,0.545147,...,0.000000,1543,712,3,0.088460,0.000000,0.122288,863464,rs13303010,pancreatic carcinoma
1,ENSG00000237613.2_1,FAM138A,lncRNA_good_csf,intergenic,lncRNA,False,lenient,0.154032,0.222609,0.465565,...,0.006667,1527,1187,3,0.000000,0.480302,0.565892,858492,rs13303010,pancreatic carcinoma
2,ENSG00000238009.6_2,RP11-34P13.7,lncRNA_good_csf,transcript_overlap,lncRNA,False,no hit,0.688701,2.415708,0.537288,...,0.505000,36987,2748,4,0.634022,0.963033,0.452706,760850,rs13303010,pancreatic carcinoma
3,ENSG00000239945.1_2,RP11-34P13.8,lncRNA_good_csf,transcript_overlap,lncRNA,False,no hit,0.027553,2.987223,0.525398,...,0.140000,1554,1319,2,0.207940,0.307150,0.113875,803468,rs13303010,pancreatic carcinoma
4,ENSG00000239906.1_1,RP11-34P13.14,lncRNA_good_csf,promoter_overlap,lncRNA,False,no hit,0.950739,19.299900,0.461300,...,0.270000,549,323,2,13.451185,15.657979,0.205026,754234,rs13303010,pancreatic carcinoma


In [68]:
supp.to_csv("../../../data/03__features/SuppTable_ScreenGenes.txt", sep="\t", index=False)

In [70]:
### look at GWAS hits
tmp = supp[supp["minimal_biotype"] == "lncRNA"]
tmp = tmp[tmp["min_hit"] == "hit"]
tmp.sort_values(by="closest_endo_snp_distance")[["gene_name", "closest_endo_snp_distance",
                                                 "closest_endo_snp_id", "closest_endo_snp_disease"]]

,gene_name,closest_endo_snp_distance,closest_endo_snp_id,closest_endo_snp_disease
6636,LINC00511,0,rs11655237,pancreatic carcinoma
3405,PVT1,0,rs17775239,lung carcinoma
2365,RP11-541P9.3,0,rs12187751,"colorectal cancer, microsatellite instability ..."
2181,RP11-414H23.2,56495,rs13355819,colorectal cancer
4054,RP11-23F23.2,193244,rs11032090,lung carcinoma
7933,RP11-120D5.1,209950,rs2788524,prostate carcinoma
7699,LINC00479,217675,rs12482500,colorectal adenoma
7252,CTC-492K19.4,238084,rs141799618,"response to radiation, prostate carcinoma"
6738,RP11-705O1.8,244193,rs62091368,"response to radiation, prostate carcinoma"
7867,RP3-508I15.9,254798,rs1014971,bladder carcinoma
